Just look to see if the sequences overlap

In [1]:
## Load any changes to local modules
%load_ext autoreload
%autoreload 2

import os
import sys

pwd = %pwd

module_path = os.path.abspath(os.path.join('{0}/../../'.format(pwd)))
if module_path not in sys.path:
    sys.path.append(module_path)

study_dir = '{}/{}'.format(module_path, 'results/03072018_proteomics_informatics_tc/')



In [2]:
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import re
import numpy as np
%matplotlib inline
from IPython.display import display

pd.set_option('precision', 5)
pd.set_option('max_colwidth', 50)



from nbcpact import AnalyzeQuantCompare,Peptide,PeptideGroup,UcbreUtils,PeptidesFromPeptideListBuilder
from dpro.protdisc import PDReaderFactory


In [ ]:


class Comparison:
    data_cache = {}
    def __init__(self,
                 nibr_input=None, 
                 ucb_input=None, 
                 ucb_output=None, 
                 desc=None, 
                 fdr=0.01):
        self.nibr_input = nibr_input
        self.ucb_input = ucb_input
        self.ucb_output = ucb_output
        self.desc = desc.copy()
        self.fdr = fdr
        
    def get_pd_reader(self):
        if not self.nibr_input in self.data_cache.keys():
            if self.nibr_input:
                if os.path.isfile(self.nibr_input):
                    factory = PDReaderFactory()
                    pdReader = factory.createPDReader(self.nibr_input,
                                                      include_non_quant=True,
                                                      fdr=self.fdr,
                                                      validate=True)
                    self.data_cache[self.nibr_input] = pdReader

        if self.nibr_input in self.data_cache.keys():
            return self.data_cache[self.nibr_input]
        else: 
            return None
    
    def get_ucb_input(self):
        if not self.ucb_input in self.data_cache.keys():
            if self.ucb_input:
                if os.path.isfile(self.ucb_input):
                    df = self.load_ip2_peptide_list(self.ucb_input)
                    self.data_cache[self.ucb_input] = df

        return self.data_cache[self.ucb_input]
    
    
    def load_ip2_peptide_list(self, file_path):
        """
            Create a DF with these cols
            ['UNIQUE_1', 'annotation', 'area_ratios', 'decoy',
           'file_AREA_MEDIAN_RATIO_1', 'ip2_peptide', 'mod_locs', 'peptide',
           'ptm_indices', 'run_counter', 'run_data', 'uniprot_ids'],

        """
        peptidesFromPeptideListBuilder = PeptidesFromPeptideListBuilder(file_path)
        df = peptidesFromPeptideListBuilder.generate_peptides_df()

        num_records = df.index.size
        num_seqs = len(set(df['peptide']))

        decoy_df = df[df['annotation'].str.contains('Reverse')]
        num_decoy_records = decoy_df.index.size
        percent_decoy_records = (num_decoy_records/num_records * 100)
        num_decoy_seqs = len(set(decoy_df['peptide']))
        percent_decoy_seqs = (num_decoy_seqs/num_seqs * 100)



        df = df[~df['annotation'].str.contains('Reverse')]
        num_forward_recs = df.index.size
        num_forward_seqs = len(set(df['peptide']))


        msg = 'Records: Tot {}, Forward {} , Decoy {} ({:.0f}%)'.format(num_records, 
                                                                   num_forward_recs, 
                                                                   num_decoy_records, 
                                                                   percent_decoy_records)
        print(msg)

        msg = 'Sequences: Tot {}, Forward {}, Decoy {} ({:.0f}%)'.format(num_seqs, 
                                                                          num_forward_seqs, 
                                                                          num_decoy_seqs, 
                                                                          percent_decoy_seqs)
        print(msg)


        df['RatioRank'] = df["file_AREA_MEDIAN_RATIO_1"].rank(ascending=True)

        ax = sns.regplot(x="RatioRank", y="file_AREA_MEDIAN_RATIO_1", data=df)
        plt.figure()

        intersection = set(df['peptide']) & set(decoy_df['peptide'])
        print('Peptide sequences seen in both the decoy and forward sets {}'.format(intersection))

        return df
    
    
        
        
def get_comparisons():

    file_path = '{}/{}'.format(study_dir, 'comparisons.txt')
    df = pd.read_table(file_path)

    comparisons = df.apply(lambda x : Comparison(nibr_input=x['NIBR_INPUT'], 
                                                 ucb_input=x['UCB_INPUT'], 
                                                 ucb_output=x['UCB_OUTPUT'], 
                                                 desc=x, 
                                                 fdr=x['FDR']), 
                           axis=1)


    #for comp in comparisons:
    #    display(comp.desc)
    return comparisons

    print('DONE')
    
comparisons = get_comparisons()

In [3]:


def load_ip2_peptide_list_old(file_path):
    
    df = pd.read_csv(file_path)
    df['ip2_sequence'] = df.sequence

    df['sequence'] = df.sequence.str.replace('\(\d+\.\d+\)','')

    num_records = df.index.size
    num_seqs = len(set(df['sequence']))

    decoy_df = df[df['protein'].str.contains('Reverse')]
    num_decoy_records = decoy_df.index.size
    percent_decoy_records = (num_decoy_records/num_records * 100)
    num_decoy_seqs = len(set(decoy_df['sequence']))
    percent_decoy_seqs = (num_decoy_seqs/num_seqs * 100)

    

    df = df[~df['protein'].str.contains('Reverse')]
    num_forward_recs = df.index.size
    num_forward_seqs = len(set(df['sequence']))
    

    msg = 'Records: Tot {}, Forward {} , Decoy {} ({:.0f}%)'.format(num_records, 
                                                               num_forward_recs, 
                                                               num_decoy_records, 
                                                               percent_decoy_records)
    print(msg)
    
    msg = 'Sequences: Tot {}, Forward {}, Decoy {} ({:.0f}%)'.format(num_seqs, 
                                                                      num_forward_seqs, 
                                                                      num_decoy_seqs, 
                                                                      percent_decoy_seqs)
    print(msg)
    
    
    df['RatioRank'] = df.AREA_MEDIAN_RATIO_1.rank(ascending=True)

    ax = sns.regplot(x="RatioRank", y="AREA_MEDIAN_RATIO_1", data=df)
    plt.figure()
    
    intersection = set(df.sequence) & set(decoy_df.sequence)
    print('Peptide sequences seen in both the decoy and forward sets {}'.format(intersection))
    
    return df
    



In [16]:
## Find overlap
def do_sequence_overlap(ip2DF=None,  pdDF=None):
    ip2DF = ip2DF.copy()
    pdDF = pdDF.copy()
    
    #display(ip2DF.head())
    #display(pdDF.head())
    
    data = {}
    pdDF['in_ip2'] = pdDF['Sequence'].isin(ip2DF['peptide'])
    ip2DF['in_pd'] = ip2DF['peptide'].isin(pdDF['Sequence'])
    
    ip2_peps = set(ip2DF['peptide'])
    pd_peps = set(pdDF['Sequence'])
    pd_peps_with_ratios = set(pdDF[~pdDF.ABUNDANCE_RATIOS.isnull()].Sequence)

    intersection = ip2_peps & pd_peps
    percent_ip2_covered = len(intersection)/len(ip2_peps)*100

      
    data['IP2 Peps'] = len(ip2_peps) 
    data['PD Peps'] = len(pd_peps)
    data['Common Peps'] = len(intersection)
    data['% IP2 Covered in PD'] = percent_ip2_covered

    intersection_with_ratios = ip2_peps & pd_peps_with_ratios
    percent_ip2_covered = len(intersection_with_ratios)/len(ip2_peps)*100

    
    overlapping_pd_peps_missing_ratios = intersection - intersection_with_ratios 
    
    
    data['Quant PD Peps'] = len(pd_peps_with_ratios)
    data['Quant Common Peps'] = len(intersection_with_ratios)
    data['% IP2 Covered with PD Quant Peps'] = percent_ip2_covered
    
    return pd.Series(data)

def do_probe_overlap(ip2DF=None,  pdDF=None):
    ip2DF = ip2DF.copy()
    pdDF = pdDF.copy()
    
    # Remove non-probed peptides
    pdDF = pdDF[pdDF['PROBE_MOD_LOCS'].apply(lambda x : len(x) > 0)]
    
    ip2DF['probe_peptide'] = ip2DF['peptide'] + '_C' + ip2DF.mod_locs.apply(lambda x : ',C'.join(map(str, x)))
    pdDF['probe_peptide'] = pdDF['Sequence'] + '_' + pdDF.PROBE_MOD_LOCS.apply(lambda x : ','.join(x))
    
    #print(pdDF['probe_peptide'])
    #print(ip2DF['probe_peptide'])
    
    data = {}
    pdDF['in_ip2'] = pdDF['probe_peptide'].isin(ip2DF['probe_peptide'])
    ip2DF['in_pd'] = ip2DF['probe_peptide'].isin(pdDF['probe_peptide'])
    
    ip2_peps = set(ip2DF['probe_peptide'])
    pd_peps = set(pdDF['probe_peptide'])
    pd_peps_with_ratios = set(pdDF[~pdDF.ABUNDANCE_RATIOS.isnull()]['probe_peptide'])

    intersection = ip2_peps & pd_peps
    percent_ip2_covered = len(intersection)/len(ip2_peps)*100
    
    data['Probed IP2 Peps'] = len(ip2_peps) 
    data['Probed PD Peps'] = len(pd_peps)
    data['Probed Common Peps'] = len(intersection)
    data['% IP2 Covered in Probed PD'] = percent_ip2_covered

    
    intersection_with_ratios = ip2_peps & pd_peps_with_ratios
    percent_ip2_covered = len(intersection_with_ratios)/len(ip2_peps)*100

    
    overlapping_pd_peps_missing_ratios = intersection - intersection_with_ratios 
    
    data['Quant PD Probed Peps'] = len(pd_peps_with_ratios)
    data['Quant Common Probed Peps'] = len(intersection_with_ratios)
    data['% IP2 Covered with PD Quant Probed Peps'] = percent_ip2_covered
    
    return pd.Series(data)
 

In [61]:
results = []
for comp in comparisons:
    if comp.get_pd_reader():
        pdDF = comp.get_pd_reader().get_target_peptides(include_additional_data=True)
        ip2DF = comp.get_ucb_input()

        parts = [comp.desc]
        result = do_sequence_overlap(ip2DF=ip2DF, pdDF=pdDF)
        parts.append(result)
        
        result = do_probe_overlap(ip2DF=ip2DF, pdDF=pdDF)
        parts.append(result)

        super_dict = {}
        for d in parts:
            for k, v in d.iteritems():  # d.items() in Python 3+
                super_dict[k] = v
                
        results.append(pd.Series(data=super_dict))
    else:
        pass
        #print('Missing PD data for {}'.format(comp.desc))
        
    
results = pd.concat(results, axis=1).T



file_path = '{}/{}'.format(study_dir, 'peptide_compare.txt')
#results.to_csv(file_path, sep='\t', index=False)

In [62]:
results

,% IP2 Covered in PD,% IP2 Covered in Probed PD,% IP2 Covered with PD Quant Peps,% IP2 Covered with PD Quant Probed Peps,Common Peps,Description,FDR,IP2 Peps,NIBR Diff Mods / Peptide*2,NIBR Enz,...,Quant Common Peps,Quant Common Probed Peps,Quant PD Peps,Quant PD Probed Peps,UCB Diff Mods / Peptide*,UCB Enz,UCB Max Internal Cleavages,UCB_FASTA,UCB_INPUT,UCB_OUTPUT
0,69.543,65.13,53.795,51.931,2813,Compare peptides with strict setting and PD FD...,0.01,4045,2,Full Tryptic,...,2176,2152,2746,2760,2,Full Tryptic,1,ip2_ip2_data_dnomura_database__UniProt_human_0...,/home/jonesmic/gBuild/jonesmic_github/proteomi...,/home/jonesmic/gBuild/jonesmic_github/proteomi...
1,69.543,65.13,53.795,51.931,2813,Compare peptides with strict setting and PD FD...,NaN,4045,2,Full Tryptic,...,2176,2152,2746,2760,2,Full Tryptic,1,ip2_ip2_data_dnomura_database__UniProt_human_0...,/home/jonesmic/gBuild/jonesmic_github/proteomi...,/home/jonesmic/gBuild/jonesmic_github/proteomi...
2,69.617,65.082,53.721,51.762,2816,Compare peptides with loose settings in PD,0.01,4045,2,Full Tryptic,...,2173,2145,2755,2770,3,Full Tryptic,1,ip2_ip2_data_dnomura_database__UniProt_human_0...,/home/jonesmic/gBuild/jonesmic_github/proteomi...,/home/jonesmic/gBuild/jonesmic_github/proteomi...


,sequence,PTM_INDEX,P-VALUE,Q-VALUE,UNIQUE_1,AREA_RATIO_ALL_1,AREA_MEDIAN_RATIO_1,AREA_RATIO_STDEV_1,AREA_RATIO_COUNT_1,LR_RATIO_1,...,XCORR_1,DEL_CN_1,LIGHT_AREA_1,HEAVY_AREA_1,PROFILE_SCORE_1,SCAN_1,ENRICHMENT_1,protein,ip2_sequence,RatioRank
0,VCEEIAIIPSKK,C35 C35 C34 C35 ;C35 C35 C35 C35 C35 C35 ;C35 ;,0.00211,0.02288,",;,,;,;","2.49666,;2.18068,3.23506,;2.15997,;",2.33867,1.20713,4,"2.31867,;1.99398,1.59523,;2.04351,;",...,"4.4906,;3.9338,4.1365,;3.5378,;","0.99494,;0.99652,0.55903,;0.99546,;","3.5683853E7,;2.75291574E8,4.1799329E7,;4.43182...","1.4292663E7,;1.26241411E8,1.2920715E7,;2.05180...","0.99235,;0.8768,0.76209,;0.98178,;","4043,;12379,14303,;13296,;","0.0,;0.0,0.0,;0.0,;",A0A075B716 40S ribosomal protein S17 OS=Homo s...,VC(464.28596)EEIAIIPSKK,6715.5
1,VCEEIAIIPSK,C35 C35 C34 C35 ;C35 C35 C35 ;C35 C35 ;,0.33989,0.84425,",;,;,,;","72.91164,;2.2837,;1.94701,2.76635,;",2.52503,5.65373,4,"32.02333,;2.07705,;1.90676,2.09064,;",...,"3.4299,;3.4779,;3.9061,3.3214,;","0.28169,;0.96581,;0.99493,0.85649,;","4.138254111E9,;1.83314464E8,;2.401761044E9,5.6...","5.6757109E7,;8.0270739E7,;1.233564624E9,2.0498...","0.83077,;0.9457,;0.94443,0.918,;","4044,;12309,;11694,15348,;","0.0,;0.0,;0.0,0.0,;",A0A075B716 40S ribosomal protein S17 OS=Homo s...,VC(470.29977)EEIAIIPSK,6925.0
2,VCEEIAIIPSKK,C35 C35 C34 C35 ;C35 C35 C35 C35 C35 C35 ;C35 ;,0.00212,0.02289,",;,,;,;","2.49666,;2.18068,3.23506,;2.15818,;",2.33867,1.20738,4,"2.31867,;1.99398,1.59523,;2.03779,;",...,"3.7909,;3.5562,3.2699,;3.5341,;","0.99494,;0.99652,0.55903,;0.99529,;","3.5683853E7,;2.75291574E8,4.1799329E7,;4.41052...","1.4292663E7,;1.26241411E8,1.2920715E7,;2.04363...","0.99235,;0.8768,0.76209,;0.98312,;","4058,;12399,14341,;13303,;","0.0,;0.0,0.0,;0.0,;",A0A075B716 40S ribosomal protein S17 OS=Homo s...,VC(470.29977)EEIAIIPSKK,6715.5
3,LWHQLTLQVLDFVQDPCFAQGDGLIK,C49 C49 C49 ;X ;X ;,1.00000,1.00000,",;X;X;","2.98859,;X;X;",2.98859,NaN,1,"1.01861,;X;X;",...,"4.5634,;X;X;","0.13808,;X;X;","4.8197971E7,;X;X;","1.6127349E7,;X;X;","0.47595,;X;X;","41572,;X;X;","0.0,;X;X;",A0A087WUL9 26S proteasome non-ATPase regulator...,LWHQLTLQVLDFVQDPC(470.29977)FAQGDGLIK,7228.0
4,IPDQLGYLVLSEGAVLASSGDLENDEQAASAISELVSTACGFR,C51 C51 C51 C51 C51 C51 ;X ;X ;,0.09789,0.31380,",,;X;X;","1.35058,1.84603,;X;X;",1.59831,1.24729,2,"1.40285,0.77465,;X;X;",...,"5.9996,7.8902,;X;X;","0.83609,0.76221,;X;X;","7427866.0,9110045.0,;X;X;","5499753.0,4934935.0,;X;X;","0.75399,0.3818,;X;X;","44095,52183,;X;X;","0.0,0.0,;X;X;",A0A087WV46 Ragulator complex protein LAMTOR4 O...,IPDQLGYLVLSEGAVLASSGDLENDEQAASAISELVSTAC(464.2...,4919.0


In [ ]:
pd_file_path = '/da/dmp/cb/jonesmic/chemgx/data/isoTopAnalysis/jonesmic/pd/pd2.2/KEA_EN80_NomiraFastaSequest_FullTryptic/KEA_EN80.pdResult'
ip2_file_path = '/home/jonesmic/gBuild/jonesmic_github/proteomics-scripts/datanocommit/peptideList.csv'


print('Compare PD on Canonical+Alternative FASTA vs. IP2')
print('{} vs. {}'.format(pd_file_path, ip2_file_path))
print('load PD')
pdReader = PDReader(pd_result_file=pd_file_path, include_non_quant=True)
pdDF = pdReader.get_target_peptides(include_additional_data=True)
print('Load IP2')
ip2DF = load_ip2_peptide_list(ip2_file_path)


do_sequence_overlap(ip2DF=ip2DF, pdDF=pdDF)   

In [ ]:
mergeDF = pd.merge(pdDF, ip2DF, left_on='Sequence', right_on='sequence')

In [ ]:
pdDF['IN_IP2'] = pdDF.Sequence.isin(ip2DF.sequence)
pdDF['NEG_LOG10_Q'] = pdDF.Qvalityqvalue.apply(lambda x : -np.log10(x+0.0001))
print(pdDF.columns)
sns.violinplot(y='NEG_LOG10_Q', x='IN_IP2', data=pdDF)

pdDF.groupby('IN_IP2')['Qvalityqvalue'].describe()
